In [1]:
import sklearn

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
train = pd.read_csv("train.csv")
# test = pd.read_csv("D:/Python/linkdinln/HateSpeechDetection/test.csv")

In [5]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
test.tail()

NameError: name 'test' is not defined

In [ ]:
sum(train["label"] == 0)

In [ ]:
sum(train["label"] == 1)

In [ ]:
train.isnull().sum()

In [7]:
!

In [8]:
import re

In [9]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [10]:
import preprocessor as p

In [183]:
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [184]:
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [191]:
# def clean_tweets(df):
#     tempArr = []
#     tmpL = p.clean(df)
#     # remove puctuation
#     tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
#     tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
#     tempArr.append(tmpL)
#     return tempArr

In [192]:
clean_tweets('#this is@ ')

['is']

In [13]:
train["clean_tweet"] = train_tweet

In [178]:
train[train['label']==1]

,id,label,tweet,clean_tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...,calls middle school build the wall chant
14,15,1,no comment! in #australia #opkillingbay #se...,no comment in
17,18,1,retweet if you agree!,retweet if you agree
23,24,1,@user @user lumpy says i am a . prove it lumpy.,lumpy says i am a prove it lumpy
34,35,1,it's unbelievable that in the 21st century we'...,its unbelievable that in the st century wed ne...
...,...,...,...,...
31934,31935,1,lady banned from kentucky mall. @user #jcpenn...,lady banned from kentucky mall
31946,31947,1,@user omfg i'm offended! i'm a mailbox and i'...,omfg im offended im a mailbox and im proud
31947,31948,1,@user @user you don't have the balls to hashta...,you dont have the balls to hashtag me as a but...
31948,31949,1,"makes you ask yourself, who am i? then am i a...",makes you ask yourself who am i then am i anyb...


In [15]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
# test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

NameError: name 'test' is not defined

In [17]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [134]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))
# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [54]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data/ training the model using fit method 
svm.fit(x_train_vec, y_train)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  94.86912086766085 %


In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(y_test, y_pred_svm)

array([[8773,  143],
       [ 349,  324]], dtype=int64)

In [24]:
from sklearn.metrics import classification_report

In [25]:
print(classification_report(y_test, y_pred_svm))


              precision    recall  f1-score   support

           0       0.96      0.98      0.97      8916
           1       0.69      0.48      0.57       673

    accuracy                           0.95      9589
   macro avg       0.83      0.73      0.77      9589
weighted avg       0.94      0.95      0.94      9589



In [161]:
text1 = 'Hello MY name is Tushar '

In [171]:
corpus = [str(text1)]
x_train_vec = vec_loaded.transform(corpus)

In [144]:
# x_train_vec = vectorizer.transform(x_train[:1])

In [158]:
import pickle

In [ ]:
filename = 'final_predict_model.sav'
pickle.dump(svm, open(filename, 'wb'))

In [165]:
filename = 'vectorize_model.sav'
pickle.dump(vectorizer, open(filename, 'wb'))

In [ ]:
import pickle
filename = 'final_predict_model.sav'
pickle.dump(svm, open(filename, 'wb'))
a vec_loaded = pickle.load(open(filename,'rb'))   
clf2 = pickle.load(open(filename,'rb'))

In [166]:
#a vec_loaded = pickle.load(open(filename,'rb'))   
# clf2 = pickle.load(open(filename,'rb'))

In [168]:
print(x_train_vec.shape)

(1, 22482)


In [169]:
x_train_vec.shape

(1, 22482)

In [172]:
clf2.predict(x_train_vec)

array([0], dtype=int64)

In [1]:
# import required libraries
import tweepy
import time
import pandas as pd
api_key ="p9btWOq38TXczri4AfCSqJjkx"
# api secret key
api_secret_key = "6eVs0lKKfp6vfnNenY8wil8PmXlVSN7g1VNIVPawG8qXQf0dLf"
# access token
access_token = "1499855933974978570-apcsJqVHjqsxOhPl4wWTIhF3L9x9lc"
# access token secret
access_token_secret = "iILNydxaee0hl8WI8ZpOHL5MUqIWg3Or53nZnyZEMfGE2"
# authorize the API Key
authentication = tweepy.OAuthHandler(api_key, api_secret_key)
# authorization to user's access token and access token secret
authentication.set_access_token(access_token, access_token_secret)
# call the api
api = tweepy.API(authentication, wait_on_rate_limit=True)

In [2]:
# tweets_for_csv = []
username = 'John'
# for tweet in tweepy.Cursor(api.user_timeline, screen_name = username).items(5):
#     print(str(tweet.full_text))
# #     tweets_for_csv.append([username, tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")])

In [5]:
for status in tweepy.Cursor(api.user_timeline,  tweet_mode='extended',screen_name=username).items(10):
    print(status.full_text)
    

RT @nelkboys: Were hot as a pistol
What a crazy week. Thank you everyone for all the support. We got an army @KyleForgeard @stevewilldoit @BobMenery @Saliimthedream.
🔥🔥 enjoy the @alienfrens event! 🖖🏼 https://t.co/u7oS2p5gs2
Who are the Nelk Boys? Why Donald Trump appeared on a frat prank podcast https://t.co/lQFYMxruz6 via @YahooNews
You’ll see these everywhere around Los Angeles… @fullsendpodcast. https://t.co/Eyezo2hvpR
@iFungibility @happydad @KyleForgeard I can’t promise it’ll be the first but DM me and I’ll make sure we show her love. Got you and your family and tell her thank you from all of us. I’m on it.
Good morning @TimJDillon 🤣🤣. https://t.co/RYBxdxvrp8
Get ready for a new https://t.co/hTmmdxmhQk. Don’t forget what I know what to build.
Crushed 😡
Pretty sad if this what you learned  from a great 53 minute interview https://t.co/ZI249hlBDU. https://t.co/D0vu4EqQS9


In [10]:
results = [status._json for status in tweepy.Cursor(API.search, q="$EURUSD", count=1000, tweet_mode='extended', lang='en').items()]
# Now you can iterate over 'results' and store the complete message from each tweet.
my_tweets = []
for result in results:
    my_tweets.append(result["full_text"])

TypeError: __init__() missing 1 required positional argument: 'method'

In [ ]:
get_related_tweets()

In [2]:
def get_related_tweets(text_query):
    # list to store tweets
    tweets_list = []
    # no of tweets
    count = 50
    try:
        # Pulling individual tweets from query
        for tweet in api.search(q=text_query, count=count):
            print(tweet.text)
            # Adding to list that contains all tweets
            tweets_list.append({'created_at': tweet.created_at,
                                'tweet_id': tweet.id,
                                'tweet_text': tweet.text})
        return pd.DataFrame.from_dict(tweets_list)
    
    except BaseException as e:
        print('failed on_status,', str(e))
#         time.sleep(3)
        


In [ ]:

import sklearn
import numpy as np
import pandas as pd
import pickle
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

import preprocessor as p

def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

train["clean_tweet"] = train_tweet

# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

from sklearn import svm

# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)
# fit the SVC model based on the given training data/ training the model using fit method 
prob = svm.fit(x_train_vec, y_train)
# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')



#Updated Code Here
#Saving SVM model 
filename = 'final_predict_model.sav'
print("Saving Modelss....")
pickle.dump(svm, open(filename, 'wb'))

#Saving Vectorization Model 

filename = 'vectorize_model.sav'
pickle.dump(vectorizer, open(filename, 'wb'))


